In [2]:
import tensorflow as tf
import numpy as np

In [5]:
# load mnist
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/")

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
# image data (feature)
print("images shape: "+ str(mnist.train.images.shape))
mnist.train.images

images shape: (55000, 784)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [8]:
# labels (note they are not one hot encoded)
mnist.train.labels

array([7, 3, 4, ..., 5, 6, 8], dtype=uint8)

In [9]:
# features (just one one dimension array of grayscale floats from 28x28 image, 784)
features = [
    tf.feature_column.numeric_column("x", shape=[mnist.train.images.shape[1]]) # shape is (784)
]

In [10]:
# number of classes (10 digits)
n_classes = len(np.unique(mnist.train.labels))

# build deep neural network classifier
# rule-of-thumb method for the number of hidden_units (hidden layers) is:
#   - The number of hidden neurons should be between the size of the input layer and the size of the output layer. (784 & 10 in this data set)
#   - The number of hidden neurons should be 2/3 the size of the input layer, plus the size of the output layer. (784*2/3 + 10 ~= 532 in this data set)
#   - The number of hidden neurons should be less than twice the size of the input layer. (532 < 784*2)
estimator = tf.estimator.DNNClassifier(feature_columns=features, n_classes=n_classes, hidden_units=[532])
estimator

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_global_id_in_cluster': 0, '_is_chief': True, '_num_ps_replicas': 0, '_session_config': None, '_keep_checkpoint_max': 5, '_task_id': 0, '_service': None, '_save_checkpoints_secs': 600, '_evaluation_master': '', '_save_checkpoints_steps': None, '_model_dir': '/var/folders/55/qkrngbr949x1jzs0c0dt70wc0000gn/T/tmpowm1gmty', '_tf_random_seed': None, '_log_step_count_steps': 100, '_num_worker_replicas': 1, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11a361e48>, '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker'}


In [11]:
# build the train function using the estimator helper function builder:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
 x={"x": mnist.train.images},
 y=mnist.train.labels.astype(np.int32),
 batch_size=50,
 shuffle=True
)

In [12]:
# train!
estimator.train(input_fn=train_input_fn, steps=200)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/55/qkrngbr949x1jzs0c0dt70wc0000gn/T/tmpowm1gmty/model.ckpt.
INFO:tensorflow:step = 1, loss = 116.02681
INFO:tensorflow:global_step/sec: 223.43
INFO:tensorflow:step = 101, loss = 15.334814 (0.450 sec)
INFO:tensorflow:Saving checkpoints for 200 into /var/folders/55/qkrngbr949x1jzs0c0dt70wc0000gn/T/tmpowm1gmty/model.ckpt.
INFO:tensorflow:Loss for final step: 6.8630238.


In [13]:
# build the test function using the estimator helper function builder:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
 x={"x": mnist.test.images},
 y=mnist.test.labels.astype(np.int32),
 num_epochs=1,
 shuffle=False
)

In [14]:
# evaluate accuracy of the trained model
accuracy_score = estimator.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-19-17:32:31
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/55/qkrngbr949x1jzs0c0dt70wc0000gn/T/tmpowm1gmty/model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-19-17:32:32
INFO:tensorflow:Saving dict for global step 200: accuracy = 0.9368, average_loss = 0.21283582, global_step = 200, loss = 26.941242

Test Accuracy: 93.680000%



In [15]:
# pick a random element in the validation set and see if we predict it correctly:
example_index = np.random.randint(len(mnist.validation.images))
example_image = mnist.validation.images[example_index].reshape(1, 784)

# predict input function
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": np.array(example_image)},
      y=None, # we are predicting y is unknown
      shuffle=False)

# predict!
predictions = estimator.predict(input_fn=predict_input_fn)

# print predictions and compare:
for prediction in list(predictions):
  print("Predicted: "+ str(prediction['class_ids']))

print("Actual label: "+ str(mnist.validation.labels[example_index]))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/55/qkrngbr949x1jzs0c0dt70wc0000gn/T/tmpowm1gmty/model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predicted: [8]
Actual label: 8
